In [1]:
import pandas as pd
import numpy as np
import re
import datetime #(per convertire il time, datetime è un modulo di pandas)
import dask.dataframe as dd
import requests
import io

In [2]:
trips= pd.read_csv(r'C:\Users\Utente\Downloads\metaData.csv')

In [3]:
#trips= pd.read_csv('https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/metaData.csv?sequence=1&isAllowed=y')

In [4]:
trips.columns

Index(['name', 'busNumber', 'startTime_iso', 'startTime_unix', 'endTime_iso',
       'endTime_unix', 'drivenDistance', 'busRoute', 'energyConsumption',
       'itcs_numberOfPassengers_mean', 'itcs_numberOfPassengers_min',
       'itcs_numberOfPassengers_max', 'status_gridIsAvailable_mean',
       'temperature_ambient_mean', 'temperature_ambient_min',
       'temperature_ambient_max'],
      dtype='object')

In [5]:
# ztb.ds= pd.read_csv('https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/ZTBus_compressed.zip?sequence=3&isAllowed=y')

In [6]:
trips

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500


1. Extract all trips with busRoute 83

In [7]:
# si può aggiungere anche la funzione loc ma non sono sicura di aver capito il perche si usa
trips[trips['busRoute'] == '83']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


2- Extract all trips where busRoute is not a number

In [8]:
# ho controllato con che minchia stessi lavorando 
trips['busRoute'].dtype # prima risultava come int64, e per poter usare una regex serve che il tipo sia una stringa

dtype('O')

In [9]:
# non sia mai che io stia tipo sbagliando tutto, per essere tranquilla trasformo 
# ma è un passagio completamente inutile
trips['busRoute1']= trips['busRoute'].astype(str)

In [10]:
# uso la regex per chiedere quali stringhe non avessere un numero nella colonna Route
trips[trips['busRoute'].str.contains(r'\D')]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,busRoute1
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15,-
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15,-
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10T03:16:11Z,1557458171,2019-05-10T18:51:37Z,1557514297,210577.00,-,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15,-
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13T03:10:23Z,1557717023,2019-05-13T23:16:13Z,1557789373,267033.80,-,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15,-
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24T02:52:47Z,1558666367,2019-05-24T22:35:11Z,1558737311,263432.60,-,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,N1,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15,N1
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,N2,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15,N2
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,N2,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15,N2
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15,N1


3. For each (busNumber, busRoute) pair, determine the number of trips

In [11]:
# QUESTA SBAGLIATAtrips.groupby(trips['busNumber'])[['busRoute','busNumber']].count()

In [12]:
# faccio così per evitare che il computer mi runni 5000 volte lo stesso valore
busNumber= trips['busNumber'].drop_duplicates()
busRoute= trips['busRoute'].drop_duplicates()

In [13]:
# creo una funzione che 'riempia' il dizionario ds con ogni elemento contenuto in 
# number e route per avere ogni possibile combo
# poi gli dico di contare quante volte compare la coppia numero x/ rotta y
ds= tot= {}
for n in busNumber:
    tot[n]={}
    for r in busRoute:
        tot[n][r]= sum(trips[(trips['busNumber']==n) & (trips['busRoute']== r)]['busNumber'])
tot        

{183: {'-': 2013,
  '31': 2196,
  '33': 23790,
  '72': 20862,
  '46': 19032,
  '32': 2196,
  '83': 80703,
  'N4': 2013,
  'N2': 3477,
  'N1': 1830},
 208: {'-': 0,
  '31': 1040,
  '33': 5200,
  '72': 9152,
  '46': 3952,
  '32': 2912,
  '83': 84240,
  'N4': 1456,
  'N2': 4160,
  'N1': 1248}}

4. For each trip, compute the ratio between the energy consumption and the average number of passengers

conversione in KiloJoules della colonna, per facilitare il calcolo

In [14]:
# qui ho convertito i joules in kilojoules perche i numeri prima facevano cagare
trips['energyConsumption(KJ)']= trips['energyConsumption']/1000

In [15]:
# ho creato la colonna con la richiesta che mi chiedeva
trips['ratio- KJ/People']= trips['energyConsumption(KJ)']/trips['itcs_numberOfPassengers_mean']
# se aggiungo la riga dopo mi da come risultato il valore intero e non i decimali, 
# converrebbe usare round valutalo con gli altri
trips['ratio- KJ/People']= trips['ratio- KJ/People'].astype(int)
trips.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,busRoute1,energyConsumption(KJ),ratio- KJ/People
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15,-,478585.2,86405
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15,31,402258.5,12147
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15,33,1445733.0,73427
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15,-,281986.7,167332
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15,72,620725.8,26131


In [16]:
#conversione anni da secondi
trips.columns

Index(['name', 'busNumber', 'startTime_iso', 'startTime_unix', 'endTime_iso',
       'endTime_unix', 'drivenDistance', 'busRoute', 'energyConsumption',
       'itcs_numberOfPassengers_mean', 'itcs_numberOfPassengers_min',
       'itcs_numberOfPassengers_max', 'status_gridIsAvailable_mean',
       'temperature_ambient_mean', 'temperature_ambient_min',
       'temperature_ambient_max', 'busRoute1', 'energyConsumption(KJ)',
       'ratio- KJ/People'],
      dtype='object')

In [17]:
# questo sarebbe utile ma esistono gia le colonne convertite
#trips['startTime_unix'] = pd.to_datetime(trips['startTime_unix'], unit='s')
#trips['endTime_unix'] = pd.to_datetime(trips['endTime_unix'], unit='s')
#trips.head()

In [18]:
# conversione kelvin celsius

In [19]:
trips['celsius.mean'] = round(trips['temperature_ambient_mean'] - 273.15)  
trips['celsius.min'] = round(trips['temperature_ambient_min'] - 273.15)
trips['celsius.max'] = round(trips['temperature_ambient_max'] - 273.15)

In [20]:
trips

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,...,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,busRoute1,energyConsumption(KJ),ratio- KJ/People,celsius.mean,celsius.min,celsius.max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,...,0.740640,282.3780,281.15,293.1500,-,478585.2,86405,9.0,8.0,20.0
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,...,0.855234,287.5443,285.15,293.1500,31,402258.5,12147,14.0,12.0,20.0
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,...,0.777860,288.7490,280.15,294.1500,33,1445733.0,73427,16.0,7.0,21.0
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,...,0.767122,282.4129,281.15,292.1500,-,281986.7,167332,9.0,8.0,19.0
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,...,0.907342,284.7325,282.15,287.1500,72,620725.8,26131,12.0,9.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.808990,...,0.739349,279.6404,278.15,291.1500,32,426041.9,10702,6.0,5.0,18.0
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.785390,...,0.434858,279.5283,278.15,292.6655,83,1536697.0,53384,6.0,5.0,20.0
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.879400,...,0.439916,279.1724,277.15,292.1500,83,1415700.0,47380,6.0,4.0,19.0
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,...,0.495739,279.4540,277.15,291.1500,N1,451916.5,22477,6.0,4.0,18.0


9. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance.

In [21]:
trips['energy/distance']= trips['energyConsumption(KJ)'] / trips['drivenDistance']

In [22]:
trips.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,...,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,busRoute1,energyConsumption(KJ),ratio- KJ/People,celsius.mean,celsius.min,celsius.max,energy/distance
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,...,282.3780,281.15,293.15,-,478585.2,86405,9.0,8.0,20.0,6.198177
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,...,287.5443,285.15,293.15,31,402258.5,12147,14.0,12.0,20.0,6.814522
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,...,288.7490,280.15,294.15,33,1445733.0,73427,16.0,7.0,21.0,6.001372
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,...,282.4129,281.15,292.15,-,281986.7,167332,9.0,8.0,19.0,6.624774
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,...,284.7325,282.15,287.15,72,620725.8,26131,12.0,9.0,14.0,4.954819


In [23]:
trips.groupby(['busNumber', 'busRoute'])['energy/distance'].sum()


busNumber  busRoute
183        -             68.219731
           31            75.595328
           32            74.134965
           33           776.767958
           46           589.402009
           72           673.747510
           83          2586.576809
           N1            59.535813
           N2           108.314257
           N4            67.407864
208        31            28.082303
           32            78.555134
           33           139.723973
           46           107.336769
           72           236.881962
           83          2332.554626
           N1            34.313737
           N2           107.698906
           N4            42.682708
Name: energy/distance, dtype: float64

In [24]:
trips.groupby(['busRoute', 'busNumber'])['energy/distance'].sum()
# non so quale delle due sia corretta per via della sintassi incapibile del professore

busRoute  busNumber
-         183            68.219731
31        183            75.595328
          208            28.082303
32        183            74.134965
          208            78.555134
33        183           776.767958
          208           139.723973
46        183           589.402009
          208           107.336769
72        183           673.747510
          208           236.881962
83        183          2586.576809
          208          2332.554626
N1        183            59.535813
          208            34.313737
N2        183           108.314257
          208           107.698906
N4        183            67.407864
          208            42.682708
Name: energy/distance, dtype: float64

10-  Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.

In [25]:
#trovo il minimo e il massimo  per coppia route/number
energy_min= trips.groupby(['busRoute', 'busNumber'])['energy/distance'].min()
energy_max= trips.groupby(['busRoute', 'busNumber'])['energy/distance'].max()

In [139]:
# li mergo insieme per creare una sorta di dataframe, poi ci creo pure una colonna
# con la differenza perche sono pazzerella e mi piace il brividino che solo 
# un nuovo dataframe mi sa dare 
energy_ds= pd.merge(energy_min, energy_max, on =['busRoute', 'busNumber'], suffixes=('_min', '_max'))
energy_ds['max-min difference']= round(energy_ds['energy/distance_max']- energy_ds['energy/distance_min'], 3)
energy_ds

energy/distance_min  energy/distance_max  \
busRoute busNumber                                             
-        183                   4.855316             7.126202   
31       183                   5.075120             8.352782   
         208                   4.779099             6.837228   
32       183                   4.718151             7.872027   
         208                   4.748898             8.224941   
33       183                   4.566781             8.135999   
         208                   4.124415             7.429116   
46       183                   4.260347             7.930619   
         208                   4.438602             6.843490   
72       183                   4.302445             8.655515   
         208                   4.302356             6.703376   
83       183                   4.326446             8.542956   
         208                   4.032769             7.924600   
N1       183                   4.756122             6.806516   
         208                   4.617548             7.589040   
N2       183                   3.932347             6.994745   
         208                   4.153241             7.401677   
N4       183                   4.894950             7.137138   
         208                   4.756114             7.102423   

                    max-min difference  
busRoute busNumber                      
-        183                     2.271  
31       183                     3.278  
         208                     2.058  
32       183                     3.154  
         208                     3.476  
33       183                     3.569  
         208                     3.305  
46       183                     3.670  
         208                     2.405  
72       183                     4.353  
         208                     2.401  
83       183                     4.217  
         208                     3.892  
N1       183                     2.050  
         208                     2.971  
N2       183                     3.062  
         208                     3.248  
N4       183                     2.242  
         208                     2.346

In [27]:
#  questo è letteralmente identico a quello che ho fatto sopra, ho solo tolto una colonna
# perche ho letto meglio la richiesta del prof... vabbe denunciatemi alla polizia dei cretini: sono colpevole
mx = trips.groupby('busRoute')[['energy/distance']].max()
mn = trips.groupby('busRoute')[['energy/distance']].min()
maxmin_df= pd.merge(mx,mn, on= 'busRoute', suffixes=('.max', '.min'))
maxmin_df['difference']= maxmin_df['energy/distance.max']- maxmin_df['energy/distance.min']
maxmin_df

,energy/distance.max,energy/distance.min,difference
busRoute,,,
-,7.126202,4.855316,2.270885
31,8.352782,4.779099,3.573683
32,8.224941,4.718151,3.506789
33,8.135999,4.124415,4.011585
46,7.930619,4.260347,3.670273
72,8.655515,4.302356,4.353159
83,8.542956,4.032769,4.510188
N1,7.589040,4.617548,2.971492
N2,7.401677,3.932347,3.469330


11-  Find the bus maximizing the difference computed in the previous point.

In [28]:
# qui ho aggiunto il commentino perche il culo del prof è una calamita per la mia lingua.
x= maxmin_df['difference'].idxmax()
y= maxmin_df['difference'].max()
print( x, 'is the bus that maximise the energy/distance ratio, with its ', round(y, 4) ,'quantity')

83 is the bus that maximise the energy/distance ratio, with its  4.5102 quantity


In [29]:
# questa parte palesemente scritta da me VA CAMBIATA??????????????
import requests
import pandas as pd
from io import BytesIO
import zipfile

# URL del file zip
url = "https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/ZTBus_samples.zip?sequence=4&isAllowed=y"

# Scarica il file zip
response = requests.get(url)
zip_file = zipfile.ZipFile(BytesIO(response.content))

# Estrai i file CSV in memoria
dfs = []
for file_name in zip_file.namelist():
    if file_name.endswith('.csv'):
        with zip_file.open(file_name) as file:
            # Leggi il file CSV in un DataFrame
            df = pd.read_csv(file)
            dfs.append(df)

# Concatena tutti i DataFrame in uno unico
details = pd.concat(dfs, ignore_index=True)


In [151]:
details

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0
1,2019-06-24T03:16:14Z,1561346174,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0
2,2019-06-24T03:16:15Z,1561346175,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0
3,2019-06-24T03:16:16Z,1561346176,-0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0
4,2019-06-24T03:16:17Z,1561346177,-0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485495,2022-11-30T19:50:18Z,1669837818,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0
485496,2022-11-30T19:50:19Z,1669837819,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0
485497,2022-11-30T19:50:20Z,1669837820,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0
485498,2022-11-30T19:50:21Z,1669837821,18.27631,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0


In [31]:
details.columns

Index(['time_iso', 'time_unix', 'electric_powerDemand', 'gnss_altitude',
       'gnss_course', 'gnss_latitude', 'gnss_longitude', 'itcs_busRoute',
       'itcs_numberOfPassengers', 'itcs_stopName',
       'odometry_articulationAngle', 'odometry_steeringAngle',
       'odometry_vehicleSpeed', 'odometry_wheelSpeed_fl',
       'odometry_wheelSpeed_fr', 'odometry_wheelSpeed_ml',
       'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable',
       'status_haltBrakeIsActive', 'status_parkBrakeIsActive',
       'temperature_ambient', 'traction_brakePressure',
       'traction_tractionForce'],
      dtype='object')

In [32]:
trips.columns

Index(['name', 'busNumber', 'startTime_iso', 'startTime_unix', 'endTime_iso',
       'endTime_unix', 'drivenDistance', 'busRoute', 'energyConsumption',
       'itcs_numberOfPassengers_mean', 'itcs_numberOfPassengers_min',
       'itcs_numberOfPassengers_max', 'status_gridIsAvailable_mean',
       'temperature_ambient_mean', 'temperature_ambient_min',
       'temperature_ambient_max', 'busRoute1', 'energyConsumption(KJ)',
       'ratio- KJ/People', 'celsius.mean', 'celsius.min', 'celsius.max',
       'energy/distance'],
      dtype='object')

5. For each station (itcs_stopName), determine the average number of passengers.

In [33]:
# si vabbe questa easy, bisonga capire se ha senso il fatto di arrotondare 
# persone intere < > media di persone

In [34]:
round(details.groupby(['itcs_stopName'])[['itcs_numberOfPassengers']].mean().dropna())

,itcs_numberOfPassengers
itcs_stopName,
"Zürich, Albisrank",18.0
"Zürich, Albisriederplatz",27.0
"Zürich, Altes Krematorium",22.0
"Zürich, Bahnhof Altstetten",10.0
"Zürich, Bahnhof Hardbrücke",28.0
...,...
"Zürich, Vogelsangstrasse",16.0
"Zürich, Waffenplatz-/Bederstr.",16.0
"Zürich, Waffenplatzstrasse",21.0


6. For each station, determine the buses that have stopped there at least once.

In [35]:
# qui bisogna usare un po di culo nel mergare

In [36]:
df= pd.merge(details, trips, left_on= 'itcs_busRoute', right_on='busRoute')


stops= df.groupby(['itcs_stopName', 'busNumber']).size().reset_index(name='count')
result= stops[stops['count'] > 0]
stops.groupby(['busNumber'])

In [43]:
df2= df[df['itcs_stopName'].str.contains(r'^(?!.*-).*$')]
df2

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,busRoute1,energyConsumption(KJ),ratio- KJ/People,celsius.mean,celsius.min,celsius.max,energy/distance
6391,2019-06-24T03:25:54Z,1561346754,34898.020,410.0000,1.200786,0.826905,0.148520,-,3.0,"Zürich, Albisriederplatz",...,282.3780,281.15,293.15,-,478585.2,86405,9.0,8.0,20.0,6.198177
6392,2019-06-24T03:25:54Z,1561346754,34898.020,410.0000,1.200786,0.826905,0.148520,-,3.0,"Zürich, Albisriederplatz",...,282.4129,281.15,292.15,-,281986.7,167332,9.0,8.0,19.0,6.624774
6393,2019-06-24T03:25:54Z,1561346754,34898.020,410.0000,1.200786,0.826905,0.148520,-,3.0,"Zürich, Albisriederplatz",...,287.5623,282.15,293.15,-,1303391.0,158361,14.0,9.0,20.0,6.189617
6394,2019-06-24T03:25:54Z,1561346754,34898.020,410.0000,1.200786,0.826905,0.148520,-,3.0,"Zürich, Albisriederplatz",...,284.6764,280.15,293.15,-,1647432.0,208756,12.0,7.0,20.0,6.169376
6395,2019-06-24T03:25:54Z,1561346754,34898.020,410.0000,1.200786,0.826905,0.148520,-,3.0,"Zürich, Albisriederplatz",...,293.1440,283.15,299.15,-,1448057.0,192554,20.0,10.0,26.0,5.496879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6909940,2022-10-26T22:55:47Z,1666824947,-5553.371,413.0719,6.003620,0.826898,0.148494,72,1.0,"Zürich, Altes Krematorium",...,287.7780,285.15,292.15,72,1050552.0,42097,15.0,12.0,19.0,5.479588
6909941,2022-10-26T22:55:47Z,1666824947,-5553.371,413.0719,6.003620,0.826898,0.148494,72,1.0,"Zürich, Altes Krematorium",...,292.3530,287.15,297.15,72,1381252.0,51391,19.0,14.0,24.0,4.751211
6909942,2022-10-26T22:55:47Z,1666824947,-5553.371,413.0719,6.003620,0.826898,0.148494,72,1.0,"Zürich, Altes Krematorium",...,292.2321,289.15,296.15,72,605042.8,28948,19.0,16.0,23.0,4.582637
6909943,2022-10-26T22:55:47Z,1666824947,-5553.371,413.0719,6.003620,0.826898,0.148494,72,1.0,"Zürich, Altes Krematorium",...,291.6134,287.15,296.15,72,1461343.0,53470,18.0,14.0,23.0,5.052321


In [38]:
stops= df2.groupby(['itcs_stopName', 'busNumber']).size().reset_index(name='count')
stops

,itcs_stopName,busNumber,count
0,"Zürich, Albisrank",183,40131
1,"Zürich, Albisrank",208,36855
2,"Zürich, Albisriederplatz",183,51610
3,"Zürich, Albisriederplatz",208,40098
4,"Zürich, Altes Krematorium",183,48863
...,...,...,...
139,"Zürich, Waffenplatzstrasse",208,2112
140,"Zürich, Wildbachstrasse",183,6517
141,"Zürich, Wildbachstrasse",208,1175
142,"Zürich, Zwinglihaus",183,5472


In [90]:
result= stops[stops['count'] >= 0]
result.groupby([ 'busNumber'], as_index= False)[['itcs_stopName']].count()



,busNumber,itcs_stopName
0,183,72
1,208,72


In [91]:
result= stops[stops['count'] >= 10]
result.groupby(['busNumber'])[['itcs_stopName']].count()

,itcs_stopName
busNumber,
183,72
208,72


15. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.

In [92]:
df.columns

Index(['time_iso', 'time_unix', 'electric_powerDemand', 'gnss_altitude',
       'gnss_course', 'gnss_latitude', 'gnss_longitude', 'itcs_busRoute',
       'itcs_numberOfPassengers', 'itcs_stopName',
       'odometry_articulationAngle', 'odometry_steeringAngle',
       'odometry_vehicleSpeed', 'odometry_wheelSpeed_fl',
       'odometry_wheelSpeed_fr', 'odometry_wheelSpeed_ml',
       'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable',
       'status_haltBrakeIsActive', 'status_parkBrakeIsActive',
       'temperature_ambient', 'traction_brakePressure',
       'traction_tractionForce', 'name', 'busNumber', 'startTime_iso',
       'startTime_unix', 'endTime_iso', 'endTime_unix', 'drivenDistance',
       'busRoute', 'energyConsumption', 'itcs_numberOfPassengers_mean',
       'itcs_numberOfPassengers_min', 'itcs_numberOfPassengers_max',
       'status_gridIsAvailable_mean', 'temperature_ambient_mean',
       'temp

df.head(7)

In [95]:
df['busNumber']= df['busNumber'].astype(str)


In [99]:

halt= df.loc[df['status_haltBrakeIsActive']==1]
park= df.loc[df['status_parkBrakeIsActive']==1]


In [124]:
park

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,busRoute1,energyConsumption(KJ),ratio- KJ/People,celsius.mean,celsius.min,celsius.max,energy/distance
0,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,282.3780,281.1500,293.15,-,478585.2,86405,9.0,8.0,20.0,6.198177
1,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,282.4129,281.1500,292.15,-,281986.7,167332,9.0,8.0,19.0,6.624774
2,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,287.5623,282.1500,293.15,-,1303391.0,158361,14.0,9.0,20.0,6.189617
3,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,284.6764,280.1500,293.15,-,1647432.0,208756,12.0,7.0,20.0,6.169376
4,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,293.1440,283.1500,299.15,-,1448057.0,192554,20.0,10.0,26.0,5.496879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296429,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,297.3314,292.1500,302.15,-,506560.5,79114,24.0,19.0,29.0,4.855316
5296430,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,294.5689,288.4197,299.15,-,1603927.0,202776,21.0,15.0,26.0,6.242282
5296431,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,300.0062,291.5750,306.15,-,1462189.0,150130,27.0,18.0,33.0,7.126202
5296432,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,301.8046,293.1500,308.15,-,1733085.0,174296,29.0,20.0,35.0,7.118117


In [118]:
halt1= halt.groupby(halt['busNumber'])[['status_haltBrakeIsActive']].count()
halt1

,status_haltBrakeIsActive
busNumber,
183,1781655
208,156820


In [154]:
halt2= park.groupby(park['busNumber'])[['status_parkBrakeIsActive']].count()
halt2

,status_parkBrakeIsActive
busNumber,
183,603823


In [127]:
halt1['status_haltBrakeIsActive'].dtype

dtype('int64')

In [135]:
final= pd.merge(halt1, halt2, on= 'busNumber', how= 'outer')
final['difference']= final['status_haltBrakeIsActive']- final['status_parkBrakeIsActive']
final

,status_haltBrakeIsActive,status_parkBrakeIsActive,difference
busNumber,,,
183,1781655,603823.0,1177832.0
208,156820,NaN,NaN


16. For each pair of stops that are consecutive in some trip, compute the average speed achieved when going from the first to the second stop.

In [136]:
trips

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,...,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,busRoute1,energyConsumption(KJ),ratio- KJ/People,celsius.mean,celsius.min,celsius.max,energy/distance
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,...,282.3780,281.15,293.1500,-,478585.2,86405,9.0,8.0,20.0,6.198177
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,...,287.5443,285.15,293.1500,31,402258.5,12147,14.0,12.0,20.0,6.814522
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,...,288.7490,280.15,294.1500,33,1445733.0,73427,16.0,7.0,21.0,6.001372
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,...,282.4129,281.15,292.1500,-,281986.7,167332,9.0,8.0,19.0,6.624774
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,...,284.7325,282.15,287.1500,72,620725.8,26131,12.0,9.0,14.0,4.954819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.808990,...,279.6404,278.15,291.1500,32,426041.9,10702,6.0,5.0,18.0,8.224941
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.785390,...,279.5283,278.15,292.6655,83,1536697.0,53384,6.0,5.0,20.0,7.316155
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.879400,...,279.1724,277.15,292.1500,83,1415700.0,47380,6.0,4.0,19.0,7.436465
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,...,279.4540,277.15,291.1500,N1,451916.5,22477,6.0,4.0,18.0,7.589040


In [149]:
df.columns


Index(['time_iso', 'time_unix', 'electric_powerDemand', 'gnss_altitude',
       'gnss_course', 'gnss_latitude', 'gnss_longitude', 'itcs_busRoute',
       'itcs_numberOfPassengers', 'itcs_stopName',
       'odometry_articulationAngle', 'odometry_steeringAngle',
       'odometry_vehicleSpeed', 'odometry_wheelSpeed_fl',
       'odometry_wheelSpeed_fr', 'odometry_wheelSpeed_ml',
       'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable',
       'status_haltBrakeIsActive', 'status_parkBrakeIsActive',
       'temperature_ambient', 'traction_brakePressure',
       'traction_tractionForce', 'name', 'busNumber', 'startTime_iso',
       'startTime_unix', 'endTime_iso', 'endTime_unix', 'drivenDistance',
       'busRoute', 'energyConsumption', 'itcs_numberOfPassengers_mean',
       'itcs_numberOfPassengers_min', 'itcs_numberOfPassengers_max',
       'status_gridIsAvailable_mean', 'temperature_ambient_mean',
       'temp

In [147]:
df2= df[df['itcs_stopName'].str.contains(r'^(?!.*-).*$')]
df2.groupby('itcs_stopName')['busNumber'].size()


itcs_stopName
Zürich, Albisrank             76986
Zürich, Albisriederplatz      91708
Zürich, Altes Krematorium     87905
Zürich, Bahnhof Altstetten    39762
Zürich, Bahnhof Hardbrücke    91863
                              ...  
Zürich, Triemli                3565
Zürich, Vogelsangstrasse       7451
Zürich, Waffenplatzstrasse     7584
Zürich, Wildbachstrasse        7692
Zürich, Zwinglihaus            7584
Name: busNumber, Length: 72, dtype: int64

In [ ]:
df_sorted = df.sort_values(by='startTime_unix')

# Calcola la differenza temporale tra le righe consecutive
df_sorted['tempo_trascorso'] = df_sorted['startTime_unix'].diff()

In [152]:
df_sorted

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,temperature_ambient_min,temperature_ambient_max,busRoute1,energyConsumption(KJ),ratio- KJ/People,celsius.mean,celsius.min,celsius.max,energy/distance,tempo_trascorso
0,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,281.1500,293.15,-,478585.2,86405,9.0,8.0,20.0,6.198177,NaN
1,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,281.1500,292.15,-,281986.7,167332,9.0,8.0,19.0,6.624774,0.0
2,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,282.1500,293.15,-,1303391.0,158361,14.0,9.0,20.0,6.189617,0.0
3,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,280.1500,293.15,-,1647432.0,208756,12.0,7.0,20.0,6.169376,0.0
4,2019-06-24T03:16:13Z,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,283.1500,299.15,-,1448057.0,192554,20.0,10.0,26.0,5.496879,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296430,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,288.4197,299.15,-,1603927.0,202776,21.0,15.0,26.0,6.242282,0.0
5296431,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,291.5750,306.15,-,1462189.0,150130,27.0,18.0,33.0,7.126202,0.0
5296432,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,293.1500,308.15,-,1733085.0,174296,29.0,20.0,35.0,7.118117,0.0
5296433,2022-11-30T19:50:22Z,1669837822,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,292.1500,299.15,-,1152757.0,198819,23.0,19.0,26.0,5.891950,0.0
